In [1]:
import glob
#import json
import re
#from pyltp import Segmentor #https://pypi.python.org/pypi/pyltp
import jieba  #https://pypi.org/project/jieba/#description
import jieba.analyse
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

path =r'C:\Users\Tse\Documents\Coursea\Big Datathon\UBS Weibo Data\weibo'
#https://blog.csdn.net/u010105243/article/details/53363416

### Import data files

In [2]:
#import Status_Data 1-8
allFiles = glob.glob(path + "/Status_Data?.json")
frame = pd.DataFrame()
list_ = []
for file_ in allFiles:
    df = pd.read_json(file_)
    list_.append(df)
df_1_8 = pd.concat(list_).drop(columns=['status_urls'])#.set_index('id')

#check duplicate
#df_1_8[df_1_8.duplicated()].shape (1430, 7)

#drop duplicate
df_1_8.drop_duplicates(keep='first', inplace=True)

#df_1_8.shape (6225, 7)

df_1_8.head()

,comments_count,created_at,id,likes_count,retweets_count,text,user_id
0,2,2015-11-05 16:40:00,D2sqd8hxB,4,9,:@世锦赛汇丰冠军赛 战罢首轮。美国公开赛黑马南非球手布兰登-格雷斯打出63杆(-9)，1杆...,1497996114
1,1,2015-11-05 14:38:00,D2rCGFUkX,3,1,:佘山4号洞，2013年@世锦赛汇丰冠军赛 冠军达斯汀-约翰逊差点上演一杆进洞！而在之前的2...,1497996114
2,1,2015-11-05 14:18:00,D2ruEc4qv,8,4,:世锦赛-汇丰冠军赛展开首日争夺，中国军团六将梁文冲、吴阿顺、李昊桐、张新军、窦泽成及曹一亮...,1497996114
3,1,2017-06-02 10:33:00,F5VQ77jUQ,5,2,:美巡#纪念高球赛# 首轮最佳球来自瑞奇-福勒18号洞果岭沙坑切球进洞，可以说十分精彩了[并...,1497996114
4,0,2017-06-02 09:54:00,F5VAappkl,4,0,:威巡REX医院公开赛首轮，@张新军golf 开局连续抓到3只小鸟，最终交出69杆，位于并列...,1497996114


In [3]:
#import Status_Data 9
df_9 = pd.read_json(path + "/"+"Status_Data9_with_city_code_new.json") 

# rename columns:  s_text -> text,  sid -> id,  uid -> user_id
df_9 = df_9.rename(columns={'s_text': 'text', 'sid': 'id', 'uid': 'user_id'}).drop(columns=['u_name'])#.set_index('id')

#no duplicate rows

#df_9.shape (4809, 7)
df_9.head()

,comments_count,created_at,likes_count,retweets_count,text,id,user_id
0,20,2018-02-28 23:59:00,16,16,:【科技独角兽IPO\'即报即审\'？10大巨头万亿市值待开发】前有360借壳今日正式上市，...,G5jCq7d7M,1988800805
1,10,2018-03-19 01:43:00,21,8,:宝骏530前一阵子刚刚公布了售价，并且外形采用了和510一样的家族眯眯眼设计，它在配置方面...,G83Zh7dWB,1895520105
2,63,2018-02-09 04:20:00,36,114,:【恒大打响2018房企大规模促销第一枪 全国楼盘88折优惠】恒大已在房企中率先打响今年大规...,G2iM478Ip,1988800805
3,1,2018-04-07 05:40:00,11,2,:#美国大师赛#15号洞(没错，就是加西亚爆出+8那一洞)，马克-利什曼成功过水两上！[给力...,GaYEQ8hI3,1497996114
4,31,2018-02-09 03:56:00,19,14,:【午评：权重跳水个股普跌 沪指重挫近6%后回升】受海外股市低迷影响，两市股指大幅低开超2....,G2iCowAOT,1988800805


### Import train label

In [4]:
class_dict={ 
0:"Stock",
1:"Bond",
2:"Oil",
3:"Gold",
4:"Real Estate",
5:"Chinese PDC",
6:"English PDC",
7:"Jewellery",
8:"Artefact",
9:"Golf",
10:"Car",
11:"Oversea Education",
12:"Yong Children Education",
}

def find_class_name(class_num):
    class_name = class_dict[class_num]
    return class_name

train_label = pd.read_table(path + "/" + 'train_label.txt', header=None, names = ["id", "dummy", "class"])

# ignore "dummy" column
train_label = train_label[["id", "class"]]

#train_label.shape (1230, 2)

train_label['class_name']=train_label['class'].apply(find_class_name)

train_label.head()

,id,class,class_name
0,zDmYsgBUj,1,Bond
1,A6xDl9OxK,10,Car
2,G9Kvsuqb0,6,English PDC
3,GfESp97V3,4,Real Estate
4,FFHya1f4H,6,English PDC


In [5]:
#check train_label vs kol df
set(train_label['id']) - set(df_9['id'])- set(df_1_8['id'])

set()

### Merge data files 1-9 and map train label

In [6]:
#Add a hierarchical index to indicate original df and name it as 'dataset'
df_master = pd.concat([df_9, df_1_8], keys=['df_9', 'df_1_8'], names=['dataset']).reset_index(level=0).sort_values('dataset')

#check dupliacte rows
#df_master[df_master.duplicated(['id'])]   (1015, 8)

df_master_w_class = pd.merge(df_master, train_label, on='id', how='left')

#df_master_w_class['kol'] = df_master_w_class['user_id'].apply(find_kol)
df_master_w_class.head()

,dataset,comments_count,created_at,id,likes_count,retweets_count,text,user_id,class,class_name
0,df_1_8,2,2013-06-04 14:53:00,zzRL6xEBV,0,5,:【鲁茨：电动汽车时代即将到来】通用汽车前副总裁鲍勃·鲁茨（Bob Lutz）认为，电动汽车...,1628009324,NaN,NaN
1,df_1_8,8,2016-09-23 14:44:00,E9Bf6obSr,22,7,:#香港蘇富比2016秋拍#台北预展将于明天在华南银行国际会议中心公开展出！超过270件总估...,2308351693,NaN,NaN
2,df_1_8,4,2016-09-23 12:04:00,E9Ac8dh2Q,29,18,:适逢常玉逝世五十周年，#香港蘇富比# 將於10月3日隆重推出「美的想象」。此乃拍卖史上最具...,2308351693,NaN,NaN
3,df_1_8,0,2016-09-23 11:19:00,E9zTG9r13,22,3,贝尔纳·布菲的妻子、其创作灵感泉源安娜贝尔（Annabel Schwob de Lure）曾...,2308351693,NaN,NaN
4,df_1_8,0,2016-09-22 19:54:00,E9tQs28z6,26,2,:纽约亚洲协会的大型回顾展「No Limits: Zao Wou-Ki」近日隆重开幕，而 #...,2308351693,NaN,NaN


### remove stop words and tokenize

In [7]:
#jieba.analyse.set_stop_words(path + "/" + "stop_word_sim_chinese.txt")
#library\jiebaRD\dict\stop_words.utf8

# 创建停用词list  
def stopwordslist(filepath):  
    stopwords = [line.strip() for line in open(filepath, 'r', encoding='utf-8').readlines()]  
    return stopwords  

# 对句子进行分词  
def seg_sentence(sentence):  
    sentence_seged = jieba.cut(sentence.strip(), cut_all=False)  
    stopwords = stopwordslist("UBS Weibo Data\weibo\stop_word_sim_chinese.txt") 
    outstr = ''  
    for word in sentence_seged:  
        if word not in stopwords:  
            if word != '\t':  
                outstr += word  
                outstr += " "  
    return outstr  

df_master_w_class['tokenized_text'] = df_master_w_class['text'].apply(seg_sentence)

#remove http links in the text
df_master_w_class['tokenized_text'] = df_master_w_class['tokenized_text'].map(lambda x: re.sub("(http://.*?\s)|(http://.*)",'',str(x)))
#group date and num into one feature
df_master_w_class['tokenized_text'] = df_master_w_class['tokenized_text'].map(lambda x: re.sub("[0-9]+",' mask_num ',str(x)))

df_master_w_class.head()

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Tse\AppData\Local\Temp\jieba.cache
Loading model cost 0.859 seconds.
Prefix dict has been built succesfully.


,dataset,comments_count,created_at,id,likes_count,retweets_count,text,user_id,class,class_name,tokenized_text
0,df_1_8,2,2013-06-04 14:53:00,zzRL6xEBV,0,5,:【鲁茨：电动汽车时代即将到来】通用汽车前副总裁鲍勃·鲁茨（Bob Lutz）认为，电动汽车...,1628009324,NaN,NaN,鲁茨 电动汽车 时代 即将 到来 通用汽车 前 副总裁 鲍勃 鲁茨 Bob Lutz 电...
1,df_1_8,8,2016-09-23 14:44:00,E9Bf6obSr,22,7,:#香港蘇富比2016秋拍#台北预展将于明天在华南银行国际会议中心公开展出！超过270件总估...,2308351693,NaN,NaN,香港 蘇 富比 mask_num 秋 拍 台北 预展 明天 华南银行 国际会议中心 公开...
2,df_1_8,4,2016-09-23 12:04:00,E9Ac8dh2Q,29,18,:适逢常玉逝世五十周年，#香港蘇富比# 將於10月3日隆重推出「美的想象」。此乃拍卖史上最具...,2308351693,NaN,NaN,适逢 常玉 逝世 五十周年 香港 蘇 富比 將於 mask_num 月 日 隆重推出...
3,df_1_8,0,2016-09-23 11:19:00,E9zTG9r13,22,3,贝尔纳·布菲的妻子、其创作灵感泉源安娜贝尔（Annabel Schwob de Lure）曾...,2308351693,NaN,NaN,贝尔纳 布菲 妻子 创作 灵感 泉源 安娜 贝尔 Annabel Schwob de...
4,df_1_8,0,2016-09-22 19:54:00,E9tQs28z6,26,2,:纽约亚洲协会的大型回顾展「No Limits: Zao Wou-Ki」近日隆重开幕，而 #...,2308351693,NaN,NaN,纽约 亚洲协会 大型 回顾展 No Limits Zao Wou Ki 近日 隆...


In [8]:
#output master file to pickle
df_master_w_class.to_pickle('df_master_w_class.pkl')

In [9]:
#line chart

In [10]:
df_train = df_master_w_class[~df_master_w_class['class'].isnull()]
df_predict = df_master_w_class[df_master_w_class['class'].isnull()]

#tabulation on dataset vs class
#pd.pivot_table(df_train, values='id', index=['dataset','class'], aggfunc=len)
#pd.pivot_table(df_train, values='class', index=['dataset'], aggfunc='count')
pd.pivot_table(df_train, values='id', index=['class','class_name'], aggfunc='count')

,,id
class,class_name,
0.0,Stock,112
1.0,Bond,58
2.0,Oil,11
3.0,Gold,10
4.0,Real Estate,46
5.0,Chinese PDC,78
6.0,English PDC,75
7.0,Jewellery,28
8.0,Artefact,241


In [11]:
"""# reading the JSON data using json.load()
file = 'Status_Data9_with_city_code_new.json'
#file = 'Status_Data9_Comment_Data_with_city_code.json'
with open(path + "/"+ file) as train_file:
    dict_train = json.load(train_file)

# converting json dataset from dictionary to dataframe
train = pd.DataFrame.from_dict(dict_train)
train.reset_index(level=0, inplace=True)

train.head()

##################################checking##################################
len(set(df_9['id']) - set(df_1_8['id']))
(set(train_label['id']) - set(df_9['id']))
#check if all comment in df_9
(set(df_comment['id']) - set(df_9['id']))
#check if df_comment['id'] appear in df_1_8
df_comment[df_comment['id'].isin(df_1_8['id'])]
##################################################
"""


"""
def jieba_tokenize(text):
    seg_list = jieba.cut(text, cut_all=False)
    tokenized_text = " ".join(seg_list)
    return tokenized_text
"""

'\ndef jieba_tokenize(text):\n    seg_list = jieba.cut(text, cut_all=False)\n    tokenized_text = " ".join(seg_list)\n    return tokenized_text\n'

In [12]:
df_master_w_class.head(20)

,dataset,comments_count,created_at,id,likes_count,retweets_count,text,user_id,class,class_name,tokenized_text
0,df_1_8,2,2013-06-04 14:53:00,zzRL6xEBV,0,5,:【鲁茨：电动汽车时代即将到来】通用汽车前副总裁鲍勃·鲁茨（Bob Lutz）认为，电动汽车...,1628009324,NaN,NaN,鲁茨 电动汽车 时代 即将 到来 通用汽车 前 副总裁 鲍勃 鲁茨 Bob Lutz 电...
1,df_1_8,8,2016-09-23 14:44:00,E9Bf6obSr,22,7,:#香港蘇富比2016秋拍#台北预展将于明天在华南银行国际会议中心公开展出！超过270件总估...,2308351693,NaN,NaN,香港 蘇 富比 mask_num 秋 拍 台北 预展 明天 华南银行 国际会议中心 公开...
2,df_1_8,4,2016-09-23 12:04:00,E9Ac8dh2Q,29,18,:适逢常玉逝世五十周年，#香港蘇富比# 將於10月3日隆重推出「美的想象」。此乃拍卖史上最具...,2308351693,NaN,NaN,适逢 常玉 逝世 五十周年 香港 蘇 富比 將於 mask_num 月 日 隆重推出...
3,df_1_8,0,2016-09-23 11:19:00,E9zTG9r13,22,3,贝尔纳·布菲的妻子、其创作灵感泉源安娜贝尔（Annabel Schwob de Lure）曾...,2308351693,NaN,NaN,贝尔纳 布菲 妻子 创作 灵感 泉源 安娜 贝尔 Annabel Schwob de...
4,df_1_8,0,2016-09-22 19:54:00,E9tQs28z6,26,2,:纽约亚洲协会的大型回顾展「No Limits: Zao Wou-Ki」近日隆重开幕，而 #...,2308351693,NaN,NaN,纽约 亚洲协会 大型 回顾展 No Limits Zao Wou Ki 近日 隆...
5,df_1_8,1,2016-09-22 18:13:00,E9tbAdH7D,23,5,:蘇富比将于香港举行「花样年华」展售会，呈献一系列十七至二十世纪中期的欧洲油画、雕塑以及精美...,2308351693,NaN,NaN,蘇 富比 香港 花样年华 展售会 呈献 一系列 十七 二十世纪 中期 欧洲 油画 雕塑 精美...
6,df_1_8,2,2016-09-21 16:40:00,E9j928wZW,12,8,:#香港蘇富比#「 瑰丽珠宝及翡翠首饰」拍卖会将于10月4日呈献一枚「克什米尔之傲」20.2...,2308351693,NaN,NaN,香港 蘇 富比 瑰丽 珠宝 翡翠 首饰 拍卖会 mask_num 月 日 呈献 一枚...
7,df_1_8,0,2016-09-21 14:14:00,E9ibUuavT,15,1,:#蘇富比# Android应用程式现已亮相，请即登陆Google Play Store h...,2308351693,NaN,NaN,蘇 富比 Android 应用程式 现已 亮相 请 登陆 Google Play ...
8,df_1_8,1,2016-09-21 14:00:00,E9i65s2gg,7,1,:夏布利(Chablis)在葡萄酒界独树一帜， Vincent Dauvissat 和 Fr...,2308351693,NaN,NaN,夏布 利 Chablis 葡萄酒 界 独树一帜 Vincent Dauvissat ...
9,df_1_8,26,2016-10-04 19:30:00,EbiOrph0H,234,64,":当代亚洲艺术两场拍卖圆满落幕，总成交额高达312,430,000港元 ！重要中国当代艺术作...",2308351693,NaN,NaN,当代 亚洲 艺术 两场 拍卖 圆满 落幕 总 成交额 高达 mask_num mask...


In [13]:
df_master_w_class.tail(20)

,dataset,comments_count,created_at,id,likes_count,retweets_count,text,user_id,class,class_name,tokenized_text
11014,df_9,3,2018-03-14 23:28:00,G7rorbGkA,9,7,:【国际早班车：在岸离岸人民币上涨 比特币跌破8000美元】国际市场行情一览：在岸、离岸人民...,1988800805,NaN,NaN,国际 早班车 在岸 离岸 人民币 上涨 比特 币 跌破 mask_num 美元 国际...
11015,df_9,30,2018-04-25 01:32:00,GdGJqpdl4,40,4,:年轻人选车真的并不简单~他们太纠结了，他们首先要经济：价格不能太贵，油耗不能太费！其次也要...,1895520105,NaN,NaN,年轻人 选车 真的 不 简单 太 纠结 经济 价格 太贵 油耗 太费 ！ 时尚 造型 立体 ...
11016,df_9,2,2018-04-02 05:10:00,GadkeaiHk,22,14,【四六级写作黄金替换80词】只要一提到“重要”脑子里闪现的单词是什么？“important”...,5563089830,NaN,NaN,四六级 写作 黄金 替换 mask_num 词 提到 脑子里 闪现 单词 importa...
11017,df_9,9,2018-03-15 11:19:00,G7w3oy7EN,9,2,:【直击315：今年哪些企业会上榜？】3月15日晚上，一年一度的“国际消费者权益日”又将声势...,1988800805,NaN,NaN,直击 mask_num 企业 会 上榜 月 mask_num 日 晚上 一年一度 国...
11018,df_9,13,2018-03-15 11:08:00,G7vYOjVF5,12,12,:【乐视网：大股东无减持计划 未形成引入投资者方案】乐视网15日晚间发布澄清公告称，近期，公...,1988800805,NaN,NaN,乐 视网 大 股东 无 减持 计划 未 引入 投资者 方案 乐 视网 mask_num...
11019,df_9,12,2018-03-15 10:11:00,G7vBN6dH5,7,6,:【海航海外资产再盘整：佳美集团IPO询价 拟募超28亿美元】海航集团旗下瑞士航空配餐公司佳...,1988800805,NaN,NaN,海航 海外 资产 再 盘整 佳美 集团 IPO 询价 拟 募超 mask_num 亿...
11020,df_9,9,2018-03-15 09:30:00,G7vkM9lus,11,6,:【马斯克要进军传媒业？公司名可能叫“砰”】人称“硅谷钢铁侠”的埃隆·马斯克，可能要进军传媒...,1988800805,NaN,NaN,马斯克 进军 传媒业 公司 名 砰 人称 硅谷 钢铁 侠 埃隆 马斯克 进军 传媒 行业 月...
11021,df_9,3,2018-03-30 07:00:00,G9LLFwoxB,17,1,:【家里橘子滞销，大三男生寝室卖果汁，同学：不含添加剂，喝着挺放心[doge]】3月27日，...,5563089830,NaN,NaN,家里 橘子 滞销 大三 男生 寝室 卖 果汁 同学 不 含 添加剂 喝 挺 放心 doge ...
11022,df_9,1,2018-03-30 02:36:00,G9K2krp3w,7,8,#公考##18省考#这些历年考过的高频成语你都知道多少？掌握这些，将逻辑填空一网打尽，迅速提...,5563089830,NaN,NaN,公考 ## mask_num 省考 历年 考过 高频 成语 都 逻辑 填空 一网打尽 提...
11023,df_9,0,2018-03-30 02:03:00,G9JP3tH6P,2,2,【雅思小作文：图表描述必备词汇】超级干货，含有最全的项目描述上升，下降，扩大等词语！转需！~...,5563089830,NaN,NaN,雅思 小 作文 图表 描述 必备 词汇 超级 干货 含有 最全 项目 描述 下降 词语 ！ ...
